## Azure ML

In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config() 

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace=ws, name="udacity-project")
run = exp.start_logging()


# to get list of al experiments
# list_experiments = Experiment.list(ws)

Workspace name: quick-starts-ws-126375
Azure region: southcentralus
Subscription id: 572f8abf-a1a0-4b78-8c6d-3630739c72b5
Resource group: aml-quickstarts-126375


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
#ws = Workspace.from_config() # get WS details

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           idle_seconds_before_scaledown=2400,
                                                           min_nodes=0,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# load data 
#datastore = ws.get_default_datastore()
#datastore.upload(src_dir='./data', target_path='datasets/cifar10', overwrite=True)
### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import *
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import shutil

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "--C": uniform(0.6, 1),
        "batch_size": choice(50, 100, 150, 200)
    }
)
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)
if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', './training')
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
              entry_script = 'train.py',
              compute_target = cpu_cluster,
              user_managed = True)### YOUR CODE HERE ###
# model = Ridge(alpha=alpha)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4,
                            estimator = est)


In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [18]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_426291ed-b91a-4b05-871e-5ae2cc57844b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_426291ed-b91a-4b05-871e-5ae2cc57844b?wsid=/subscriptions/572f8abf-a1a0-4b78-8c6d-3630739c72b5/resourcegroups/aml-quickstarts-126375/workspaces/quick-starts-ws-126375

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-15T23:43:44.605544][API][INFO]Experiment created<END>\n""<START>[2020-11-15T23:43:45.296780][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2020-11-15T23:43:45.049949][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-11-15T23:43:46.3925512Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_426291ed-b91a-4b05-871e-5ae2cc57844b
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_426291ed-b91a-4b05-871e-5ae2cc57844b?wsid=/subscriptions/572f8

{'runId': 'HD_426291ed-b91a-4b05-871e-5ae2cc57844b',
 'target': 'cpu-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-11-15T23:43:44.381305Z',
 'endTimeUtc': '2020-11-15T23:52:20.108083Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": {\n      "type": null,\n      "message": "[(https://southcentralus.experiments.azureml.net/subscriptions/572f8abf-a1a0-4b78-8c6d-3630739c72b5/resourceGroups/aml-quickstarts-126375/providers/Microsoft.Machin

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [5]:
assert(hyperdrive_run.get_status() == "Completed")

AssertionError: 

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

# parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

# save file
os.makedirs('./outputs',exist_ok = True)

joblib.dump(value=best_run, filename = 'outputs/bestHDModel.txt')

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
#define model
model = best_run.register_model(model_name='model_project', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# 
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path = url)
### YOUR CODE HERE ###

In [ ]:
import pandas as pd
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)   ### YOUR DATA OBJECT HERE ###)

In [ ]:
# Split data into train and test sets.
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
ds = ws.get_default_datastore()
ds.upload(src_dir='training/', target_path='bankmarketing', overwrite=True, show_progress=True)

In [ ]:

# train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
label = "y"
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=X,
    label_column_name=y,
    n_cross_validations=5)
'''
automl_config = AutoMLConfig(task="classification",
                             X=your_training_features,
                             y=your_training_labels,
                             iterations=30,
                             iteration_timeout_minutes=5,
                             primary_metric="AUC_weighted",
                             n_cross_validations=5
                            )
'''

In [2]:
# Submit your automl run
from azureml.widgets import RunDetails

### YOUR CODE HERE ###

pipeline_run = exp.submit(automl_config,show_output = True)
pipeline_run.wait_for_completion(show_output=True)

# run = experiment.submit(config=automl_config, show_output=True)
'''

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_customized, fitted_model  = remote_run.get_output()
print(best_run_customized)

In [ ]:
best_run_customized.register_model(model_name = "automlproject.pkl", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
#cpu_cluster.delete()

In [ ]:
# Retrieve and save your best automl model.

joblib.dump(value=fitted_model, filename='outputs/bestautomlmodel.py')

In [ ]:
compute_target.delete()